자막 토픽 모델링

In [ ]:
#코랩 드라이브 연동
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd 
df=pd.read_csv('gdrive/My Drive/텍마팀플/DAJUNG_vlog_subtitle2.csv')

In [ ]:
import re
han = re.compile(r'[^ ㄱ-ㅣ가-힣]+')
text2=[]
for text in df['subtitle2']:
  text_list = text[3:-2].split(', ')
  for i in text_list:
    tokens = re.sub(han,"",i)
    # stopwords
    tokens = re.sub("진짜|다정|여기|올해|오늘|정도|영상|시간|생각|조금|다시|동동|선택|자고|계속|제가|요즘|여러분","",tokens)
    tokens.strip()
    text2.append(tokens.strip())

In [ ]:
! pip install konlpy

     |████████████████████████████████| 19.4MB 16.3MB/s 
     |████████████████████████████████| 460kB 26.3MB/s 
     |████████████████████████████████| 92kB 9.2MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import konlpy
from konlpy.tag import Kkma, Okt, Hannanum, Komoran
kkma = Kkma()
okt = Okt()
hannanum = Hannanum()
komoran = Komoran()

In [ ]:
#word2vec
vec = []
for i in text2:
    vec2 = []
    tm_ls = hannanum.nouns(i)
    if len(tm_ls) == 0: #비어있는 리스트 삭제
        pass
    else:
        for j in range(len(tm_ls)): #비어있지 않은 리스트 중에서 단어가 한 음절인 것은 pass
            if len(tm_ls[j]) > 1:
                vec2.append(tm_ls[j])
            else:
                pass
    if len(vec2) == 0:
        pass
    else:
        vec.append(vec2)

In [ ]:
tt = pd.DataFrame({'tokens':vec})
#역토큰화 필요
tokenized_doc = tt['tokens']
detokenized_doc = []
for i in range(len(tt)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
tt['tokens'] = detokenized_doc
tt

,tokens
0,밑그림
1,앨리스 완성
2,퍼팅 시작 이유
3,좋을까 고민
4,이것저것 적성
...,...
2112,책임
2113,도전 궁금
2114,현실 안주 변화
2115,변화 그것


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(tt['tokens'])
X.shape # TF-IDF 행렬의 크기 확인

(2117, 1000)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=20,learning_method='online',random_state=777,max_iter=1)
lda_model.fit_transform(X) #토픽별 단어들 분포
print(lda_model.components_)
print(lda_model.components_.shape) 

[[0.12910532 0.12612843 0.1413372  ... 0.13650402 0.15161991 0.12773089]
 [0.11776341 0.12968437 0.12913238 ... 0.13628274 0.13151054 0.13539386]
 [0.1285933  0.15445115 0.13063287 ... 0.13106565 0.14693546 0.13006769]
 ...
 [0.1465944  0.1420289  0.12851811 ... 0.13647123 0.1406241  0.13090681]
 [0.13733194 0.13665168 0.14427291 ... 0.13519374 0.13157353 0.13159579]
 [0.13334498 0.12369529 0.14336757 ... 0.41532761 0.12514855 0.14975217]]
(20, 1000)


In [ ]:
# hannanum
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=15):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('식물들', 10.9), ('이거', 9.7), ('걱정', 6.57), ('공간', 5.01), ('변화', 4.15), ('현실', 3.49), ('조립', 3.06), ('감정', 2.98), ('안주', 2.94), ('발견', 2.78), ('건강', 2.77), ('오래간만', 2.72), ('자랑', 2.71), ('그것', 2.68), ('비하인드스토리', 2.49)]
Topic 2: [('준비', 13.15), ('고북', 11.47), ('당신', 11.39), ('마지막', 9.68), ('때문', 8.72), ('음식', 7.07), ('지금', 4.52), ('식사', 3.8), ('손질', 3.54), ('차분', 3.28), ('포인트', 3.16), ('치즈', 3.04), ('이걸', 2.99), ('사이', 2.81), ('식탁', 2.81)]
Topic 3: [('창문', 7.35), ('소금', 6.18), ('사실', 4.95), ('동안', 4.76), ('이곳', 4.13), ('이상', 3.21), ('스틱', 2.64), ('부분', 2.62), ('ㅎㅎ', 2.59), ('바게트', 2.45), ('먹기', 2.42), ('홀더', 2.38), ('이번', 2.36), ('이센스', 2.36), ('신경', 2.17)]
Topic 4: [('청소', 19.52), ('뭔가', 15.01), ('시작', 14.13), ('하루', 11.5), ('사용', 9.91), ('고민', 7.16), ('택배', 6.07), ('퇴근', 5.18), ('완료', 4.23), ('다이어리', 3.51), ('의욕', 3.02), ('냄새', 2.83), ('보리굴비', 2.64), ('도전', 2.48), ('무기력', 2.3)]
Topic 5: [('바닥', 10.8), ('이불', 9.05), ('정신', 8.81), ('보여드릴게요', 7.53), ('저희', 6.47), ('분위기', 5.27), (

In [ ]:
# hannanum
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=15):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('감사', 17.24), ('시작', 14.41), ('하나', 13.91), ('하루', 11.64), ('식물들', 10.9), ('이불', 10.49), ('기분', 10.38), ('이야기', 8.18), ('베란다', 8.13), ('고민', 7.02), ('걱정', 6.71), ('깨끗', 6.25), ('택배', 6.21), ('간단', 5.41), ('딸기', 4.89)]
Topic 2: [('청소', 19.63), ('뭔가', 14.25), ('구입', 14.1), ('고북', 14.07), ('퇴사', 12.17), ('점심', 11.58), ('마지막', 9.71), ('다이어트', 9.41), ('토마토', 8.26), ('창문', 7.43), ('소리', 6.39), ('자수', 6.28), ('두부', 4.83), ('평소', 4.64), ('지금', 4.62)]
Topic 3: [('아침', 16.88), ('저녁', 16.13), ('준비', 13.21), ('당신', 11.49), ('바닥', 10.86), ('영화', 9.32), ('다음', 9.23), ('때문', 9.21), ('것들', 7.85), ('요구르트', 7.77), ('일주일', 7.74), ('음식', 7.14), ('저희', 6.6), ('식사', 5.64), ('카페', 5.49)]
Topic 4: [('만나요', 11.51), ('샐러리', 10.84), ('엄마', 10.36), ('사용', 10.2), ('완성', 9.95), ('이거', 9.82), ('주방', 7.74), ('그릇', 7.65), ('운동', 7.24), ('촬영', 6.82), ('경우', 6.44), ('엽서', 6.29), ('침실', 5.63), ('불안', 5.62), ('직업', 5.56)]
Topic 5: [('크리스마스', 17.54), ('선물', 15.53), ('겨울', 11.84), ('사람', 10.43), ('정리', 10.14), ('

In [ ]:
# okt
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=15):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('겨울', 13.25), ('선물', 13.22), ('갑자기', 11.33), ('그냥', 10.76), ('마지막', 9.28), ('다이어트', 9.07), ('토마토', 8.68), ('바닥', 7.97), ('추천', 7.85), ('직업', 7.04), ('주방', 6.83), ('엽서', 6.19), ('자수', 5.99), ('요구르트', 5.8), ('촬영', 5.75)]
Topic 2: [('청소', 17.74), ('구입', 12.21), ('이제', 10.67), ('공부', 10.53), ('매일', 10.45), ('사실', 10.26), ('사용', 9.98), ('완성', 9.76), ('개월', 7.23), ('카메라', 7.23), ('베란다', 7.22), ('마무리', 6.29), ('소금', 6.15), ('당근', 4.95), ('경우', 4.87)]
Topic 3: [('크리스마스', 17.13), ('뭔가', 15.04), ('시작', 14.47), ('정리', 11.58), ('점심', 10.87), ('하루', 10.39), ('그릇', 10.11), ('기분', 10.02), ('정말', 9.73), ('지금', 8.89), ('영화', 8.88), ('거의', 8.45), ('원래', 7.61), ('일주일', 7.56), ('달력', 7.18)]
Topic 4: [('하나', 18.75), ('아침', 15.39), ('준비', 14.73), ('식물', 13.83), ('당신', 12.0), ('다음', 10.98), ('이불', 10.55), ('가장', 10.18), ('바로', 8.05), ('고민', 7.27), ('이야기', 7.13), ('창문', 6.96), ('침대', 6.71), ('걱정', 6.68), ('사람', 6.62)]
Topic 5: [('저녁', 15.1), ('이건', 13.35), ('보고', 11.88), ('엄마', 9.96), ('맥주', 8.5), 

In [ ]:
# okt
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=15):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('겨울', 13.36), ('그냥', 10.69), ('토마토', 8.66), ('바닥', 7.89), ('마무리', 7.26), ('직업', 7.01), ('추천', 6.72), ('사람', 6.66), ('주방', 6.43), ('트리', 6.31), ('도전', 6.23), ('엽서', 6.17), ('소리', 5.63), ('활동', 5.29), ('고민', 4.14)]
Topic 2: [('뭔가', 14.9), ('준비', 14.82), ('사용', 10.52), ('완성', 9.65), ('가지', 7.35), ('음식', 6.52), ('재료', 5.33), ('자꾸', 4.17), ('의욕', 4.05), ('다이어리', 4.04), ('한낮', 4.02), ('그날', 3.9), ('눈사람', 3.71), ('손질', 3.49), ('식사', 3.4)]
Topic 3: [('기분', 9.93), ('일주일', 7.36), ('베란다', 7.2), ('달력', 7.13), ('창문', 7.07), ('냉동', 4.72), ('행주', 4.57), ('우리', 4.43), ('처음', 4.36), ('첫날', 4.06), ('가끔', 3.92), ('테이프', 3.81), ('이유', 3.47), ('기록', 3.33), ('습관', 3.28)]
Topic 4: [('하나', 15.72), ('다이어트', 9.05), ('잔뜩', 4.87), ('경우', 4.77), ('광고', 4.42), ('용도', 4.11), ('간장', 4.0), ('고등어', 3.75), ('고민', 3.68), ('운동', 3.66), ('냄새', 3.62), ('어서', 3.62), ('보시', 3.47), ('과일', 3.32), ('순간', 2.83)]
Topic 5: [('크리스마스', 17.07), ('정리', 11.51), ('점심', 11.0), ('이제', 10.77), ('정말', 9.5), ('맥주', 8.46), ('때문', 7.

In [ ]:
# kkma
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=15):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('구입', 12.56), ('식물', 12.27), ('습관', 9.69), ('토마토', 9.55), ('사실', 8.78), ('베란다', 7.87), ('달력', 6.95), ('이야기', 6.94), ('침실', 5.37), ('개월', 5.32), ('걱정', 5.19), ('처음', 4.97), ('거실', 3.59), ('우리', 3.44), ('평소', 3.38)]
Topic 2: [('선물', 17.49), ('청소', 16.12), ('고민', 15.71), ('도전', 14.52), ('준비', 10.83), ('반복', 9.94), ('이거', 9.05), ('요구르트', 8.13), ('이제', 5.32), ('음식', 5.14), ('러리', 4.95), ('재료', 4.52), ('간장', 3.93), ('노래', 3.64), ('녹색', 3.49)]
Topic 3: [('안정', 18.84), ('저녁', 14.37), ('불안', 11.07), ('사용', 6.97), ('마무리', 6.71), ('트리', 5.96), ('자수', 5.71), ('카페', 5.52), ('커튼', 4.64), ('양파', 4.41), ('편집', 4.36), ('다이어리', 4.04), ('눈사람', 3.61), ('작년', 3.04), ('불편', 2.94)]
Topic 4: [('감사', 15.31), ('사람', 7.43), ('원래', 7.28), ('나의', 6.75), ('맥주', 6.44), ('공간', 5.98), ('소리', 5.7), ('장갑', 5.59), ('촬영', 5.25), ('수프', 4.68), ('커피', 4.3), ('광고', 3.86), ('직업', 3.74), ('무엇', 3.51), ('테이프', 3.02)]
Topic 5: [('내가', 19.11), ('도전', 14.61), ('이것', 12.39), ('때문', 7.04), ('카메라', 6.6), ('정신', 5.26), ('딸기

In [ ]:
# komoran
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=15):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('크리스마스', 17.9), ('아침', 16.24), ('다음', 12.61), ('정리', 12.41), ('식사', 8.96), ('요구르트', 8.83), ('베란다', 8.62), ('일주일', 8.49), ('마무리', 7.42), ('일정', 7.13), ('소리', 6.2), ('자수', 6.02), ('분위기', 5.07), ('편집', 4.81), ('용도', 4.44)]
Topic 2: [('시작', 14.27), ('하루', 13.3), ('이건', 13.26), ('엄마', 10.61), ('토마토', 8.92), ('원래', 7.79), ('택배', 6.52), ('혼자', 5.97), ('장갑', 5.71), ('새해', 5.49), ('운동', 5.01), ('넷플릭스', 4.54), ('의욕', 4.41), ('케이크', 4.24), ('샐러드', 3.88)]
Topic 3: [('감사', 17.31), ('사실', 10.38), ('다이어트', 9.11), ('공부', 7.32), ('불안', 5.03), ('딸기', 4.58), ('필요', 4.27), ('첫날', 3.81), ('이유', 3.68), ('설거지', 3.63), ('마늘', 3.22), ('토요일', 3.12), ('책임', 3.02), ('정성', 3.02), ('수제비', 2.88)]
Topic 4: [('준비', 14.02), ('이불', 9.92), ('음식', 6.44), ('카페', 5.63), ('처음', 5.57), ('보니', 5.19), ('양파', 4.43), ('차려', 4.33), ('녹색', 4.11), ('재료', 4.06), ('과일', 3.75), ('빈티지', 3.73), ('데코', 3.54), ('손질', 3.37), ('아파트', 3.31)]
Topic 5: [('청소', 18.65), ('침대', 6.18), ('계획', 6.01), ('오리', 5.78), ('변화', 4.76), ('당근', 4.7

In [ ]:
# komoran
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('크리스마스', 17.73), ('아침', 16.33), ('식물', 14.78), ('시작', 14.29), ('하루', 13.39), ('정리', 12.73), ('다음', 12.67), ('점심', 12.29), ('겨울', 11.55), ('식사', 10.29)]
Topic 2: [('이건', 13.33), ('엄마', 11.09), ('퇴사', 10.18), ('공부', 9.84), ('영화', 8.84), ('마음', 7.85), ('개월', 7.63), ('그림', 7.19), ('운동', 6.96), ('소금', 5.75)]
Topic 3: [('감사', 17.37), ('완성', 11.46), ('사용', 10.72), ('사실', 10.48), ('이야기', 8.56), ('정신', 7.58), ('그릇', 7.44), ('달력', 7.17), ('오리', 6.51), ('촬영', 5.85)]
Topic 4: [('저녁', 17.97), ('준비', 14.03), ('이불', 11.67), ('마지막', 10.29), ('다이어트', 9.02), ('때문', 8.74), ('사진', 7.51), ('카메라', 7.47), ('고민', 7.26), ('음식', 6.51)]
Topic 5: [('청소', 18.73), ('선물', 17.43), ('구입', 13.13), ('기분', 10.91), ('창문', 6.65), ('계획', 6.08), ('분위기', 5.14), ('직업', 5.06), ('트리', 4.94), ('당근', 4.82)]
